# Importy

In [1]:
from awpy import DemoParser
from awpy.analytics.states import generate_vector_state
from tqdm.notebook import tqdm
import pandas as pd
import patoolib
import os
import shutil
import warnings
warnings.filterwarnings("ignore")

# Funkcje do wyciągania feature'ów

In [2]:
def get_team_data(frame, team, mapping):
    team_frame = frame[team]
    team_data = {}
    team_data[team + 'Name'] = team_frame['teamName']
    team_data[team + 'EqVal'] = team_frame['teamEqVal']
    team_data[team + 'AlivePlayers'] = team_frame['alivePlayers']
    team_data[team + 'TotalUtility'] = team_frame['totalUtility']
    
    for player in team_frame['players']:
        mapped_player = mapping[player['steamID']]
        for key_player in player:
            if key_player != 'inventory':
                team_data[f'{team}{mapped_player}_{key_player}'] = player[key_player]
            elif key_player == 'inventory':
                team_data[f"{team}{mapped_player}_Smoke Grenade"] = 0 
                team_data[f"{team}{mapped_player}_Flashbang"] = 0
                team_data[f"{team}{mapped_player}_Decoy Grenade"] = 0
                team_data[f"{team}{mapped_player}_fireGrenades"] = 0
                team_data[f"{team}{mapped_player}_HE Grenade"] = 0
                if player[key_player] is None:
                    team_data[f'{team}{mapped_player}_mainWeapon'] = ''
                    team_data[f'{team}{mapped_player}_secondaryWeapon'] = ''
                else:
                    for weapon in player[key_player]:
                        if weapon['weaponClass'] == 'Pistols':
                            team_data[f'{team}{mapped_player}_secondaryWeapon'] = weapon['weaponName']
                        elif weapon['weaponClass'] == 'Grenade':
                            if weapon['weaponName'] in {"Molotov", "Incendiary Grenade"}:
                                team_data[f"{team}{mapped_player}_fireGrenades"] = weapon['ammoInMagazine'] + weapon['ammoInReserve']
                            else:
                                team_data[f"{team}{mapped_player}_{weapon['weaponName']}"] = weapon['ammoInMagazine'] + weapon['ammoInReserve']
                        else:
                            team_data[f'{team}{mapped_player}_mainWeapon'] = weapon['weaponName']
                    if f'{team}{mapped_player}_mainWeapon' not in team_data and f'{team}{mapped_player}_secondaryWeapon' not in team_data:
                        team_data[f'{team}{mapped_player}_mainWeapon'] = ''
                    elif f'{team}{mapped_player}_mainWeapon' not in team_data:
                        team_data[f'{team}{mapped_player}_mainWeapon'] = team_data[f'{team}{mapped_player}_secondaryWeapon']
    return team_data

def get_frame_data(frame, mapping):
    frame_data = {**get_team_data(frame, 'ct', mapping), **get_team_data(frame, 't', mapping)}
    frame_data['bombPlanted'] = frame['bombPlanted']
    frame_data['bombsite'] = frame['bombsite']
    frame_data['tick'] = frame['tick']
    frame_data['seconds'] = frame['seconds']
    frame_data['clockTime'] = frame['clockTime']
    bomb_data = frame['bomb']
    for key in bomb_data:
        frame_data[f"bomb_{key}"] = bomb_data[key]
    return frame_data


def create_mapping(round_):
    ct_players = round_['ctSide']
    map_steam_id = {}
    for i, player in enumerate(ct_players['players']):
        map_steam_id[player['steamID']] = f'Player_{i + 1}'
    
    t_players = round_['tSide']
    for i, player in enumerate(t_players['players']):
        map_steam_id[player['steamID']] = f'Player_{i + 1}'
    
    return map_steam_id

In [3]:
def get_match_data(data):
    data_list = []
    mapping = create_mapping(data['gameRounds'][0])
    for round_ in data['gameRounds']:
        for frame in round_['frames']:
            converted_vector = get_frame_data(frame, mapping)
            converted_vector['roundNum'] = round_['roundNum']
            data_list.append(converted_vector)
        last_tick = round_['endTick']
    res = pd.DataFrame(data_list)
    res.fillna(method='ffill', inplace=True)
    return res

# Konfiguracja folderów

In [4]:
directory = r'../Demka' # Katalog z demkami
temp_directory = r'csgo_tmp' # Tymczasowy katalog z przetwarzanym meczem

In [5]:
if os.path.exists(temp_directory):
    shutil.rmtree(temp_directory)

# Przejście po meczach

In [6]:
all_matches = []
for file in tqdm(os.listdir(directory), desc="Match", leave=True, position=0):
    patoolib.extract_archive(os.path.join(directory, file), outdir=temp_directory)
    for demo in tqdm(os.listdir(temp_directory)[0:40], desc="Map", leave=False):
        demo_parser = DemoParser(
            demofile = os.path.join(temp_directory, demo), 
            demo_id = demo[:-4],
            parse_rate=32, 
            trade_time=5, 
            buy_style="hltv"
        )
        parsed_demo = demo_parser.parse()
        match_df = get_match_data(parsed_demo)
        all_matches.append(match_df)
        os.remove(demo[:-4] + ".json")
    shutil.rmtree(temp_directory)

Match:   0%|          | 0/72 [00:00<?, ?it/s]

patool: Extracting ../Demka\PGL-Major-Antwerp-2022-bad-news-eagles-vs-imperial-bo3_2356153.rar ...
patool: running "C:\Program Files\7-Zip\7z.EXE" x -ocsgo_tmp -- ../Demka\PGL-Major-Antwerp-2022-bad-news-eagles-vs-imperial-bo3_2356153.rar
patool: ... ../Demka\PGL-Major-Antwerp-2022-bad-news-eagles-vs-imperial-bo3_2356153.rar extracted to `csgo_tmp'.


Map:   0%|          | 0/2 [00:00<?, ?it/s]

A parse rate lower than 64 may be slow depending on the tickrate of the demo, which is usually 64 for MM and 128 for pro demos.
A parse rate lower than 64 may be slow depending on the tickrate of the demo, which is usually 64 for MM and 128 for pro demos.


patool: Extracting ../Demka\PGL-Major-Antwerp-2022-big-vs-furia-vertigo_2356149.rar ...
patool: running "C:\Program Files\7-Zip\7z.EXE" x -ocsgo_tmp -- ../Demka\PGL-Major-Antwerp-2022-big-vs-furia-vertigo_2356149.rar
patool: ... ../Demka\PGL-Major-Antwerp-2022-big-vs-furia-vertigo_2356149.rar extracted to `csgo_tmp'.


...

Map:   0%|          | 0/2 [00:00<?, ?it/s]

A parse rate lower than 64 may be slow depending on the tickrate of the demo, which is usually 64 for MM and 128 for pro demos.
A parse rate lower than 64 may be slow depending on the tickrate of the demo, which is usually 64 for MM and 128 for pro demos.


patool: Extracting ../Demka\PGL-Major-Antwerp-2022-vitality-vs-outsiders-mirage_2356142.rar ...
patool: running "C:\Program Files\7-Zip\7z.EXE" x -ocsgo_tmp -- ../Demka\PGL-Major-Antwerp-2022-vitality-vs-outsiders-mirage_2356142.rar
patool: ... ../Demka\PGL-Major-Antwerp-2022-vitality-vs-outsiders-mirage_2356142.rar extracted to `csgo_tmp'.


Map:   0%|          | 0/1 [00:00<?, ?it/s]

A parse rate lower than 64 may be slow depending on the tickrate of the demo, which is usually 64 for MM and 128 for pro demos.


In [7]:
merged_df = pd.concat(all_matches)
merged_df.to_parquet('player_data.parquet.gzip', compression='gzip')

In [8]:
df = pd.read_parquet('player_data.parquet.gzip')
df

,ctName,ctEqVal,ctAlivePlayers,ctTotalUtility,ctPlayer_2_steamID,ctPlayer_2_name,ctPlayer_2_team,ctPlayer_2_side,ctPlayer_2_x,ctPlayer_2_y,...,tPlayer_3_zoomLevel,bombPlanted,bombsite,tick,seconds,clockTime,bomb_x,bomb_y,bomb_z,roundNum
0,Bad News Eagles,4100,5,0,76561198234835452,gxx-,Bad News Eagles,CT,2287.882324,2019.412842,...,0.0,False,,911,0.125000,01:55,-1686.543457,484.882690,-61.769085,1
1,Bad News Eagles,4100,5,0,76561198234835452,gxx-,Bad News Eagles,CT,2266.610107,1977.265137,...,0.0,False,,943,0.375000,01:55,-1655.148804,460.171936,-63.327736,1
2,Bad News Eagles,4100,5,0,76561198234835452,gxx-,Bad News Eagles,CT,2249.704834,1924.318359,...,0.0,False,,975,0.625000,01:55,-1618.272583,416.995819,-64.436058,1
3,Bad News Eagles,4100,5,0,76561198234835452,gxx-,Bad News Eagles,CT,2207.921631,1887.511963,...,0.0,False,,1007,0.875000,01:55,-1575.204834,374.794678,-66.670403,1
4,Bad News Eagles,4100,5,0,76561198234835452,gxx-,Bad News Eagles,CT,2151.189209,1862.758057,...,0.0,False,,1039,1.125000,01:54,-1532.967651,341.931915,-66.130608,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9177,Outsiders,30150,1,1,76561198127236290,Qikert,Outsiders,CT,-736.572754,-2503.424561,...,0.0,True,A,383373,19.921260,00:21,-272.218750,-2155.843750,-175.343750,24
9178,Outsiders,30150,1,1,76561198127236290,Qikert,Outsiders,CT,-736.572754,-2503.424561,...,0.0,True,A,383405,20.173228,00:20,-272.218750,-2155.843750,-175.343750,24
9179,Outsiders,30150,1,1,76561198127236290,Qikert,Outsiders,CT,-736.572754,-2503.424561,...,0.0,True,A,383437,20.425197,00:20,-272.218750,-2155.843750,-175.343750,24
9180,Outsiders,30150,1,1,76561198127236290,Qikert,Outsiders,CT,-736.572754,-2503.424561,...,0.0,True,A,383469,20.677165,00:20,-272.218750,-2155.843750,-175.343750,24


# Check danych

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1242425 entries, 0 to 9181
Columns: 517 entries, ctName to roundNum
dtypes: bool(181), float64(186), int64(76), object(74)
memory usage: 3.3+ GB


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ctEqVal,1242425.0,2.381138e+04,8.820094e+03,1.000000e+03,2.130000e+04,2.755000e+04,3.000000e+04,3.575000e+04
ctAlivePlayers,1242425.0,3.854691e+00,1.395288e+00,0.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00,5.000000e+00
ctTotalUtility,1242425.0,5.461514e+00,5.006568e+00,0.000000e+00,1.000000e+00,4.000000e+00,8.000000e+00,2.000000e+01
ctPlayer_2_steamID,1242425.0,7.656120e+16,1.965545e+08,7.656120e+16,7.656120e+16,7.656120e+16,7.656120e+16,7.656120e+16
ctPlayer_2_x,1242425.0,-2.611313e+02,1.373396e+03,-3.959999e+03,-1.351887e+03,-3.426783e+02,7.820131e+02,3.498000e+03
...,...,...,...,...,...,...,...,...
seconds,1242425.0,4.014980e+01,2.880594e+01,0.000000e+00,1.643307e+01,3.440945e+01,5.978740e+01,1.460079e+02
bomb_x,1242425.0,-3.590560e+02,1.157639e+03,-3.959999e+03,-1.253614e+03,-3.606562e+02,5.701562e+02,2.634000e+03
bomb_y,1242425.0,-2.147858e+02,1.235652e+03,-3.400931e+03,-1.083375e+03,-2.583391e+02,6.016562e+02,3.512206e+03
bomb_z,1242425.0,1.671678e+03,4.144178e+03,-7.759688e+02,-1.659688e+02,3.002116e+01,1.600312e+02,1.198500e+04


In [11]:
df_desc = df.describe().T
df_desc[df_desc['count'] != 1242425.0]

,count,mean,std,min,25%,50%,75%,max
ctPlayer_4_z,1242417.0,1715.421367,4184.247739,-775.96875,-165.96875,89.951195,171.978867,11964.197266


In [20]:
df.loc[df.ctPlayer_4_z.isna(), df.columns[df.columns.str.startswith('ctPlayer_4')]]

,ctPlayer_4_steamID,ctPlayer_4_name,ctPlayer_4_team,ctPlayer_4_side,ctPlayer_4_x,ctPlayer_4_y,ctPlayer_4_z,ctPlayer_4_velocityX,ctPlayer_4_velocityY,ctPlayer_4_velocityZ,...,ctPlayer_4_equipmentValueFreezetimeEnd,ctPlayer_4_equipmentValueRoundStart,ctPlayer_4_cash,ctPlayer_4_cashSpendThisRound,ctPlayer_4_cashSpendTotal,ctPlayer_4_hasHelmet,ctPlayer_4_hasDefuse,ctPlayer_4_hasBomb,ctPlayer_4_ping,ctPlayer_4_zoomLevel
0,76561198165327895,sinnopsyy,Bad News Eagles,CT,2467.358154,2086.319092,NaN,-34.922691,-113.908531,0.0,...,950,200,150,650,650,False,False,False,5,0
1,76561198165327895,sinnopsyy,Bad News Eagles,CT,2454.257324,2043.637695,NaN,-66.179916,-214.269150,0.0,...,950,200,150,650,650,False,False,False,5,0
2,76561198165327895,sinnopsyy,Bad News Eagles,CT,2436.208496,1985.858887,NaN,-75.526756,-238.318497,0.0,...,950,200,150,650,650,False,False,False,5,0
3,76561198165327895,sinnopsyy,Bad News Eagles,CT,2416.770752,1924.406372,NaN,-74.775276,-238.555374,0.0,...,950,200,150,650,650,False,False,False,5,0
4,76561198165327895,sinnopsyy,Bad News Eagles,CT,2398.150879,1864.744263,NaN,-74.300400,-238.703690,0.0,...,950,200,150,650,650,False,False,False,5,0
5,76561198165327895,sinnopsyy,Bad News Eagles,CT,2379.546631,1805.077637,NaN,-74.752846,-238.562393,0.0,...,950,200,150,650,650,False,False,False,5,0
6,76561198165327895,sinnopsyy,Bad News Eagles,CT,2362.048584,1745.091431,NaN,-58.982037,-242.942627,0.0,...,950,200,150,650,650,False,False,False,5,0
7,76561198165327895,sinnopsyy,Bad News Eagles,CT,2351.114502,1685.562988,NaN,-36.046459,-247.387650,0.0,...,950,200,150,650,650,False,False,False,5,0


In [25]:
df.loc[df.ctPlayer_4_z.isna(), df.columns[df.columns.str.endswith(('_x','_y','_z'))]]

,ctPlayer_2_x,ctPlayer_2_y,ctPlayer_2_z,ctPlayer_1_x,ctPlayer_1_y,ctPlayer_1_z,ctPlayer_4_x,ctPlayer_4_y,ctPlayer_4_z,ctPlayer_3_x,...,tPlayer_4_z,tPlayer_5_x,tPlayer_5_y,tPlayer_5_z,tPlayer_3_x,tPlayer_3_y,tPlayer_3_z,bomb_x,bomb_y,bomb_z
0,2287.882324,2019.412842,128.031250,2406.093262,2359.477051,128.03125,2467.358154,2086.319092,NaN,2463.019531,...,-65.566566,-1546.392456,720.161194,-51.337925,-1686.543457,484.882690,-61.769085,-1686.543457,484.882690,-61.769085
1,2266.610107,1977.265137,128.031250,2376.409912,2394.062500,128.03125,2454.257324,2043.637695,NaN,2482.533203,...,-66.133408,-1542.945679,675.883423,-49.052753,-1655.148804,460.171936,-63.327736,-1655.148804,460.171936,-63.327736
2,2249.704834,1924.318359,128.031250,2336.987061,2439.998047,128.03125,2436.208496,1985.858887,NaN,2476.192627,...,-63.877693,-1541.271484,618.583313,-43.968750,-1618.272583,416.995819,-64.436058,-1618.272583,416.995819,-64.436058
3,2207.921631,1887.511963,128.031250,2296.283936,2487.427734,128.03125,2416.770752,1924.406372,NaN,2460.610107,...,-61.773483,-1551.063599,560.372864,-63.775856,-1575.204834,374.794678,-66.670403,-1575.204834,374.794678,-66.670403
4,2151.189209,1862.758057,128.031250,2255.088867,2534.416504,128.03125,2398.150879,1864.744263,NaN,2433.229248,...,-63.968750,-1549.699097,497.938171,-63.968750,-1532.967651,341.931915,-66.130608,-1532.967651,341.931915,-66.130608
5,2092.306641,1841.811035,128.031250,2205.211670,2571.833740,128.03125,2379.546631,1805.077637,NaN,2428.998291,...,-59.975212,-1549.159668,436.944672,-66.340149,-1512.703979,304.416290,-63.067162,-1512.703979,304.416290,-63.067162
6,2033.106567,1821.770752,151.481583,2150.758545,2602.493164,128.03125,2362.048584,1745.091431,NaN,2402.999268,...,-63.968750,-1545.333130,375.400543,-67.516357,-1462.986572,290.134979,-60.902756,-1462.986572,290.134979,-60.902756
7,1975.349365,1799.306152,160.031250,2095.008057,2630.368164,128.03125,2351.114502,1685.562988,NaN,2365.573975,...,-63.968750,-1546.992798,314.372131,-65.388214,-1436.327148,272.031616,-60.385487,-1436.327148,272.031616,-60.385487


In [28]:
df.loc[df.ctPlayer_4_z.isna(), 'clockTime']

0    01:55
1    01:55
2    01:55
3    01:55
4    01:54
5    01:54
6    01:54
7    01:54
Name: clockTime, dtype: object

Braki danych, bo najpewniej gracze wyszli z serwera po przegranym meczu (przynajmniej u mnie tak było).